In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/598-DLH/mimic
%ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/598-DLH/mimic
ADMISSIONS.csv.gz          D_ICD_DIAGNOSES.csv.gz     NOTEEVENTS.csv.gz
CALLOUT.csv.gz             D_ICD_PROCEDURES.csv.gz    OUTPUTEVENTS.csv.gz
CAREGIVERS.csv.gz          D_ITEMS.csv.gz             PATIENTS.csv.gz
CHARTEVENTS.csv.gz         D_LABITEMS.csv.gz          PRESCRIPTIONS.csv.gz
checksum_md5_unzipped.txt  DRGCODES.csv.gz            PROCEDUREEVENTS_MV.csv.gz
checksum_md5_zipped.txt    ICUSTAYS.csv.gz            PROCEDURES_ICD.csv.gz
cleaned_text.pkl           INPUTEVENTS_CV.csv.gz      README.md
CPTEVENTS.csv.gz           INPUTEVENTS_MV.csv.gz      SERVICES.csv.gz
DATETIMEEVENTS.csv.gz      LABEVENTS.csv.gz           SHA256SUMS.txt
D_CPT.csv.gz               LICENSE.txt                TRANSFERS.csv.gz
DIAGNOSES_ICD.csv.gz       MICROBIOLOGYEVENTS.csv.gz


# Read dataframe

In [5]:
import pandas as pd
READ_PROPORTION_FOR_NOTES = 30
notes = pd.read_csv('NOTEEVENTS.csv.gz',skiprows = [i for i in range(1, 2083180*(READ_PROPORTION_FOR_NOTES-1) // READ_PROPORTION_FOR_NOTES) ], compression='gzip') #only 1619465 has charttime

# Text (for TF-IDF and BERT)

In [6]:
notes.columns = notes.columns.str.lower()

## Functions to clean text

In [7]:
import re
import nltk
from gensim.parsing.preprocessing import remove_stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
def remove_date(text):
    return re.sub(r'\d{4}\-\d{1,2}\-\d{1,2}', ' ', text)

def remove_special_characters(text):
    text = re.sub(r'[^a-zA-Z0-9-]', ' ', text)
    ## Remove '-' but keep negative  (i.e. 'sign-in' => 'sign in', '------', => '' , '-123' => '-123') 
    text = re.sub(r'(\s+)(\-)(\d+)', r'\1#\3', text)
    text = re.sub(r'\-',' ', text)
    text = re.sub(r'#', '-', text)
    return text

def remove_multiple_spaces(text):
    return re.sub('\s+', ' ', text).strip()

def process_text_into_words(text):
    text = remove_date(text)
    text = remove_special_characters(text)
    text = remove_multiple_spaces(text.lower())
    text = remove_stopwords(text)
    words = word_tokenize(text)
    return words

def process_text_into_word_lists(text):
    sentences = text.split('\n')
    output_sentence = list(map(process_text_into_words, sentences))
    output_sentence = list(filter(None, output_sentence))
    return output_sentence

In [9]:
%%time
import itertools
notes['text'] = notes['text'].map(process_text_into_word_lists).reset_index(drop=True)
notes = notes.drop(columns=['row_id', 'storetime','category','description', 'cgid']).reset_index(drop=True)
notes['text_bert'] = notes['text'].map(lambda sentences: [' '.join(sentence) for sentence in sentences])
notes['text_tfidf']= notes['text'].map(lambda sentences: list(itertools.chain(*sentences)))
notes = notes.drop(columns=['text']).reset_index(drop=True)
notes.to_pickle('cleaned_text.pkl')

CPU times: user 3min 26s, sys: 2.33 s, total: 3min 28s
Wall time: 3min 29s


In [10]:
%ls

ADMISSIONS.csv.gz          D_ICD_DIAGNOSES.csv.gz     NOTEEVENTS.csv.gz
CALLOUT.csv.gz             D_ICD_PROCEDURES.csv.gz    OUTPUTEVENTS.csv.gz
CAREGIVERS.csv.gz          D_ITEMS.csv.gz             PATIENTS.csv.gz
CHARTEVENTS.csv.gz         D_LABITEMS.csv.gz          PRESCRIPTIONS.csv.gz
checksum_md5_unzipped.txt  DRGCODES.csv.gz            PROCEDUREEVENTS_MV.csv.gz
checksum_md5_zipped.txt    ICUSTAYS.csv.gz            PROCEDURES_ICD.csv.gz
cleaned_text.pkl           INPUTEVENTS_CV.csv.gz      README.md
CPTEVENTS.csv.gz           INPUTEVENTS_MV.csv.gz      SERVICES.csv.gz
DATETIMEEVENTS.csv.gz      LABEVENTS.csv.gz           SHA256SUMS.txt
D_CPT.csv.gz               LICENSE.txt                TRANSFERS.csv.gz
DIAGNOSES_ICD.csv.gz       MICROBIOLOGYEVENTS.csv.gz
